In [1]:
from io import BytesIO
import urllib.request
from zipfile import ZipFile
import time
import pandas as pd
import os
import dask.dataframe as dd
from datetime import date, datetime, timedelta

In [ ]:
def ptrans(anomes):
    ini = time.time()
    url = f'https://portaldatransparencia.gov.br/download-de-dados/auxilio-emergencial/{anomes}'
    with urllib.request.urlopen(url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(f"{anomes}")
    fim = time.time()
    print(f"Arquivo do mês {anomes} baixado, duração: {fim-ini}")

In [2]:
meses = [202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012]

In [ ]:
for mes in meses:
    ptrans(mes)

In [3]:
def path(anomes):
    endereco = "C:/Users/grama/Desktop/TCC MBA/dash_python/"
    final = "_AuxilioEmergencial.csv"
    resultado = endereco + str(anomes) + '/' + str(anomes) + final 
    return resultado

In [4]:
cols = ['mes_disp', 'uf', 'enquadramento', 'valor']

In [7]:
for mes in meses:
    aux = dd.read_csv(path(mes), names=cols, sep = ';', usecols=[0, 1, 10, 13], encoding = 'latin1', skiprows=1, dtype={'uf': 'object', 'valor': 'float64'}, decimal = ',')
    var = aux.groupby(['mes_disp', 'uf', 'enquadramento'])['valor'].agg(['sum', 'count']).compute()
    globals()['sumarizado_' + str(mes)] = var

In [8]:
dados = [sumarizado_202004, sumarizado_202005, sumarizado_202006, sumarizado_202007, sumarizado_202008, sumarizado_202009, sumarizado_202010, sumarizado_202011, sumarizado_202012]

In [11]:
juncao = pd.concat(dados, axis=0, ignore_index=False)

juncao.to_csv('dados_auxilio_sumarizados.csv')

In [ ]:
juncao.head()

In [15]:
juncao['sum'].sum()

293339694300.0

In [14]:
juncao['count'].sum()

533441034